## Limitations of **Batch Normalization**:
- You need to maintain running means
- Tricky for RNNs. Do you need normalization for each step?
- Doens't work with small batch sizes
- Need to compute means and variances across devices in distributed training

## Layer Normalization
- Layer Normalization is a simpler normalization method that works on a wider range of settings
- It transforms the inputs to have zero mean and variance **across all features** 
- Note: BN fixes the zero mean and unit variance **for each element**
- Layer normalization does it for each batch across all elements!